In [ ]:

# Load the Drive helper and mount
from google.colab import drive
import os

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dir = os.listdir('drive/MyDrive/datasets/Corn')
print(dir)

['Validation', 'Training', 'Testing']


In [ ]:
dir = os.listdir('drive/MyDrive/datasets/Corn/Training')
print(dir)

['blight', 'grey_leaf_spot', 'healthy', 'common_rust']


In [ ]:
dir = os.listdir('drive/MyDrive')

print(dir)

['mnist_test.csv', 'mnist_train.csv', 'Colab Notebooks', 'Apple Leaf Disease', 'model', 'saved_sedensenet121_saved.keras', 'sedensenet121_after_part7.keras', 'sedensenet121_after_part8.keras', 'sedensenet121_after_part9.keras', 'sedensenet121_after_part10.keras', 'sedensenet121_after_part11.keras', 'datasets', 'best_custom_model1_part1.keras', 'inception_frozen_part1.keras', 'best_epoch90_inception_frozen_part1.keras', 'max_planck_weather_ts_modified.csv', 'best_epoch23_densenet121_part2.keras', 'Copy of best_epoch16_densenet121_part1.keras', 'Training_A3', 'Saved Models Apples', 'Saved Models Corn']


In [ ]:
import tensorflow as tf
import imageio
import numpy as np


def read_images(dir):

    supported_extensions = [".jpg", ".jpeg", ".png"]
    image_list = []
    count = 0
    # Walk through the directory and read images
    for root, _, files in os.walk(dir):
        for file in files:
            file_extension = os.path.splitext(file)[-1].lower()

            # Check if the file is a .jpg or .jpeg image
            if file_extension in supported_extensions:
                image_path = os.path.join(root, file)
                try:
                    image = imageio.imread(image_path)
                    image1 = image

                    image = np.asarray(image)
                    del image1
                    image_list.append(image)
                except Exception as e:
                    print(f"Error reading image {image_path}: {e}")

            count += 1

            if count % 100 == 0:
              print(str(count) + " images read")


    return image_list



In [ ]:
# @title
# Train data directories
base_train_dir = 'drive/MyDrive/datasets/Corn/Training'
base_test_dir = 'drive/MyDrive/datasets/Corn/Validation'


# Train data directories
blight_train = read_images(os.path.join(base_train_dir, 'blight'))
grey_leaf_spot_train = read_images(os.path.join(base_train_dir, 'grey_leaf_spot'))
healthy_train = read_images(os.path.join(base_train_dir, 'healthy'))
common_rust_train = read_images(os.path.join(base_train_dir, 'common_rust'))

# Test data directories
blight_test = read_images(os.path.join(base_test_dir, 'blight'))
grey_leaf_spot_test = read_images(os.path.join(base_test_dir, 'grey_leaf_spot'))
healthy_test = read_images(os.path.join(base_test_dir, 'healthy'))
common_rust_test = read_images(os.path.join(base_test_dir, 'common_rust'))



<ipython-input-5-2dac8990de03>:20: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


100 images read
200 images read
300 images read
400 images read
500 images read
600 images read
700 images read
100 images read
200 images read
300 images read
100 images read
200 images read
300 images read
400 images read
500 images read
600 images read
700 images read
100 images read
200 images read
300 images read
400 images read
500 images read
600 images read
700 images read
800 images read
100 images read
100 images read
100 images read


In [ ]:
# For training classes
print("Length of blight_train:", len(blight_train))
print("Length of grey_leaf_spot_train:", len(grey_leaf_spot_train))
print("Length of healthy_train:", len(healthy_train))
print("Length of common_rust_train:", len(common_rust_train))

# For test classes
print("Length of blight_test:", len(blight_test))
print("Length of grey_leaf_spot_test:", len(grey_leaf_spot_test))
print("Length of healthy_test:", len(healthy_test))
print("Length of common_rust_test:", len(common_rust_test))

# Calculate the lengths of all train parts
train_lengths = len(blight_train) + len(grey_leaf_spot_train) + len(healthy_train) + len(common_rust_train)

# Calculate the lengths of all test parts
test_lengths = len(blight_test) + len(grey_leaf_spot_test) + len(healthy_test) + len(common_rust_test)

# Print the results
print("Total length of all train parts:", train_lengths)
print("Total length of all test parts:", test_lengths)

# Calculate the total length of all parts
total_length = train_lengths + test_lengths

# Print the results
print("Total length of all parts:", total_length)


Length of blight_train: 779
Length of grey_leaf_spot_train: 390
Length of healthy_train: 790
Length of common_rust_train: 888
Length of blight_test: 137
Length of grey_leaf_spot_test: 69
Length of healthy_test: 139
Length of common_rust_test: 156
Total length of all train parts: 2847
Total length of all test parts: 501
Total length of all parts: 3348


In [ ]:
import imgaug as ia
import imgaug.augmenters as iaa
import random
import copy

def add_gaussian_noise(images, mean_range=(0, 15), std_range=(0, 0.15)):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.AdditiveGaussianNoise(loc=mean_range, scale=(0, 0.2*255))
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images

def random_crop(images, crop_percent=(0.1, 0.4)):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.Crop(percent=crop_percent)
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images

def random_rotate(images, rotation_range=(-360, 360)):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.Rotate(rotate=rotation_range)
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images

def invert_images(images):
    ia.seed(1)
    # Define the augmentation pipeline
    seq = iaa.Sequential([
        iaa.Add(value=(-20, 20))
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    # images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images]

    return augmented_images


def adjust_brightness(images, brightness_range=(-65, 65)):
    ia.seed(1)
    # Define the augmentation pipeline for adjusting brightness
    seq = iaa.Sequential([
        iaa.Multiply((1.0 + brightness_range[0] / 100.0, 1.0 + brightness_range[1] / 100.0))
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images_np]

    return augmented_images


def scale_images(images, scale_factor = (0.3, 1.8)):
    ia.seed(1)
    # Define the augmentation pipeline for scaling images
    seq = iaa.Sequential([
        iaa.Affine(scale=scale_factor)
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images_np]

    return augmented_images

def add_contrast(images, contrast_factor=(0.5, 1.5)):
    ia.seed(1)
    # Define the augmentation pipeline for adding contrast
    seq = iaa.Sequential([
        iaa.ContrastNormalization(alpha=contrast_factor)
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images_np]

    return augmented_images

def flip_images(images, flip_probability=0.5):
    ia.seed(1)
    # Define the augmentation pipeline for randomly flipping images
    seq = iaa.Sequential([
        iaa.Sometimes(flip_probability, iaa.Fliplr(1.0)),  # Horizontal flips
        iaa.Sometimes(flip_probability, iaa.Flipud(1.0))   # Vertical flips
    ])

    # Convert images to numpy array (imgaug requires numpy arrays)
    images_np = np.array(images)

    # Perform augmentation on each image individually
    augmented_images = [seq(image=image) for image in images_np]

    return augmented_images


# discard images according to ratio
def discard_images(images, discard_ratio=0.5):
    random.seed(10)
    # Calculate the number of images to discard based on the discard_ratio
    num_images_to_discard = int(len(images) * discard_ratio)

    # Create a copy of the input list to avoid modifying the original list
    remaining_images = images[:]

    # Randomly discard a portion of the images
    random.shuffle(remaining_images)
    remaining_images = remaining_images[num_images_to_discard:]

    return remaining_images




def resize_images(images_list, width=128, height=128):
    ia.seed(1)
    # Define the resize augmentation
    resize_augmenter = iaa.Resize({"height": height, "width": width})

    resized_images = []

    for image in images_list:
        # Ensure the image is in RGB format (imgaug uses RGB by default)
        if image.shape[-1] == 1:  # Grayscale image with single channel
            image = np.repeat(image, 3, axis=-1)

        # Apply the resize augmentation
        augmented_image = resize_augmenter.augment_image(image)

        # Append the augmented image to the result list
        resized_images.append(augmented_image)

    del images_list[:]
    return resized_images

def keep_n_images(images, n_to_keep):
    random.seed(10)
    if n_to_keep >= len(images):
        return images  # Keep all images if n_to_keep is greater than or equal to the image count

    # Randomly shuffle the images list
    random.shuffle(images)

    # Keep the first n_to_keep images and discard the rest
    kept_images = images[:n_to_keep]

    # Create a copy of the kept images list
    kept_images_copy = copy.deepcopy(kept_images)

    # Clear the original images list to free memory
    del images[:]

    return kept_images_copy
import cv2
def normalize_images(image_list):

  for i in range(len(image_list)):
      image = image_list[i].astype(np.float32) / 255.0
      image_list[i] = image

def discard_images(lst, percent_to_discard):
    # Randomly select images to discard using NumPy
    num_images_to_discard = int(len(lst) * percent_to_discard)
    indices_to_discard = np.random.choice(len(lst), size=num_images_to_discard, replace=False)

    # Create a new list without the discarded images
    modified_lst = np.delete(lst, indices_to_discard, axis=0)

    return list(modified_lst)


In [ ]:

blight_all = (
    add_gaussian_noise(blight_train) +
    random_crop(blight_train) +
    invert_images(blight_train) +
    adjust_brightness(blight_train) +
    scale_images(blight_train) +
    random_rotate(blight_train) +
    add_contrast(blight_train) +
    flip_images(blight_train) +
    scale_images(blight_train) +
    random_rotate(blight_train)
    )



grey_leaf_spot_all = (
    add_gaussian_noise(grey_leaf_spot_train) +
    random_crop(grey_leaf_spot_train) +
    invert_images(grey_leaf_spot_train) +
    adjust_brightness(grey_leaf_spot_train) +
    scale_images(grey_leaf_spot_train) +
    random_rotate(grey_leaf_spot_train) +
    add_contrast(grey_leaf_spot_train) +
    flip_images(grey_leaf_spot_train)  +
    scale_images(grey_leaf_spot_train) +
    random_rotate(grey_leaf_spot_train)

)

healthy_all = (
    add_gaussian_noise(healthy_train) +
    random_crop(healthy_train) +
    invert_images(healthy_train) +
    adjust_brightness(healthy_train) +
    scale_images(healthy_train) +
    random_rotate(healthy_train) +
    add_contrast(healthy_train) +
    flip_images(healthy_train) +
    scale_images(healthy_train) +
    random_rotate(healthy_train)
    )


common_rust_all = (
    add_gaussian_noise(common_rust_train) +
    random_crop(common_rust_train) +
    invert_images(common_rust_train) +
    adjust_brightness(common_rust_train) +
    scale_images(common_rust_train) +
    random_rotate(common_rust_train) +
    add_contrast(common_rust_train) +
    flip_images(common_rust_train) +
    scale_images(common_rust_train) +
    random_rotate(common_rust_train)
  )


<ipython-input-8-162926356415>:75: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images_np = np.array(images)
<ipython-input-8-162926356415>:91: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images_np = np.array(images)
/usr/local/lib/python3.10/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
<ipython-input-8-162926356415>:106: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is

In [ ]:
blight_train = blight_train + blight_all
grey_leaf_spot_train = grey_leaf_spot_train + grey_leaf_spot_all
healthy_train = healthy_train + healthy_all
common_rust_train = common_rust_train + common_rust_all


In [ ]:
import numpy as np

labels_blight_train = np.zeros(len(blight_train))
labels_grey_leaf_spot_train = np.ones(len(grey_leaf_spot_train))
labels_healthy_train = np.full(len(healthy_train), 2)
labels_common_rust_train = np.full(len(common_rust_train), 3)

# Combine train labels
labels_train = np.concatenate([
    labels_blight_train,
    labels_grey_leaf_spot_train,
    labels_healthy_train,
    labels_common_rust_train
])

labels_blight_test = np.zeros(len(blight_test))
labels_grey_leaf_spot_test = np.ones(len(grey_leaf_spot_test))
labels_healthy_test = np.full(len(healthy_test), 2)
labels_common_rust_test = np.full(len(common_rust_test), 3)

# Combine test labels
labels_test = np.concatenate([
    labels_blight_test,
    labels_grey_leaf_spot_test,
    labels_healthy_test,
    labels_common_rust_test
])


In [ ]:
images_train = blight_train + grey_leaf_spot_train + healthy_train + common_rust_train
images_test = blight_test + grey_leaf_spot_test + healthy_test + common_rust_test


In [ ]:
images_train = resize_images(images_train, 128,128)
images_test = resize_images(images_test, 128, 128)

In [ ]:

normalize_images(images_train)



In [ ]:
images_train[80]

array([[[0.5176471 , 0.5176471 , 0.5254902 ],
        [0.5294118 , 0.5254902 , 0.54901963],
        [0.5294118 , 0.52156866, 0.5568628 ],
        ...,
        [0.59607846, 0.58431375, 0.6117647 ],
        [0.57254905, 0.56078434, 0.5882353 ],
        [0.49411765, 0.48235294, 0.50980395]],

       [[0.5254902 , 0.5254902 , 0.53333336],
        [0.5254902 , 0.52156866, 0.54509807],
        [0.5294118 , 0.5176471 , 0.5529412 ],
        ...,
        [0.5803922 , 0.5686275 , 0.59607846],
        [0.5921569 , 0.5803922 , 0.60784316],
        [0.5686275 , 0.5568628 , 0.5803922 ]],

       [[0.52156866, 0.52156866, 0.5294118 ],
        [0.52156866, 0.50980395, 0.5372549 ],
        [0.5294118 , 0.5176471 , 0.56078434],
        ...,
        [0.5882353 , 0.5764706 , 0.6117647 ],
        [0.59607846, 0.58431375, 0.62352943],
        [0.5882353 , 0.5764706 , 0.6117647 ]],

       ...,

       [[0.41568628, 0.40392157, 0.43137255],
        [0.4392157 , 0.42745098, 0.45490196],
        [0.42745098, 0

In [ ]:
normalize_images(images_test)

In [ ]:
#TEST Normalization

In [ ]:
print(len(images_train))
print(len(labels_train))

counts = [0,0,0,0]

for label in labels_train:
    counts[int(label)] += 1

counts

31317
31317


[8569, 4290, 8690, 9768]

In [ ]:

ls1 = images_test
ls2 = images_train

images_test = np.array(images_test)
images_train = np.array(images_train)

del ls1[:]
del ls2[:]

shuffle_indices_train = np.random.permutation(len(images_train))
shuffle_indices_test = np.random.permutation(len(images_test))


images_train = images_train[shuffle_indices_train]
labels_train = labels_train[shuffle_indices_train]

images_test = images_test[shuffle_indices_test]
labels_test = labels_test[shuffle_indices_test]

In [ ]:
# %load custom_callback.py
import tensorflow as tf
import numpy as np
import pandas as pd

# Find accuracy of model
def find_accuracy(test,pred):
    correct = 0
    total = len(test)

    for i in range(len(test)):
        if test[i] == pred[i]:
            correct += 1

    return correct/total


# Map ANN outputs to classes
def get_labels(y_pred_ann):
    labels = []

    for pred in y_pred_ann:
        max_index = 0

        for i in range(len(pred)):
            if pred[i] > pred[max_index]:
                max_index = i

        labels.append(max_index)

    return labels

# This callback prints accuracy by epoch information after each epoch
class Save_Accuracy_By_Epoch(tf.keras.callbacks.Callback):
    def __init__(self, test_data):
        self.X_Test = test_data[0]
        self.Y_Test = test_data[1]
        self.accuracies = []
        self.epochs = []

    def on_epoch_end(self, epoch, logs = None):
        y_pred = self.model.predict(self.X_Test)

        if epoch == 4:
            pass

        y_pred = get_labels(y_pred)
        accuracy = find_accuracy(self.Y_Test, y_pred)
        self.epochs.append(epoch+1)
        self.accuracies.append(accuracy)



        print(self.epochs)
        print(self.accuracies)


# This callback prints metrics for every class after each epoch
class Save_Multiclass_Metrics_By_Epoch(tf.keras.callbacks.Callback):
    def __init__(self, test_data, n_classes, save_after = 10, save_csv_path = 'results.csv', model_name = 'model.keras'):
        self.X_Test = test_data[0]
        self.Y_Test = test_data[1]
        self.epochs = []
        self.n_classes = n_classes
        self.save_after = save_after
        self.save_csv_path = save_csv_path
        self.model_name = model_name
        self.max_accuracy = 0

        self.mat_sensitivity = []
        self.mat_specificity = []
        self.mat_precision = []
        self.mat_recall = []
        self.mat_accuracy = []
        self.mat_f1 = []
        self.accuracies = []

        for i in range(n_classes):
            self.mat_sensitivity.append([])
            self.mat_specificity.append([])
            self.mat_precision.append([])
            self.mat_recall.append([])
            self.mat_accuracy.append([])
            self.mat_f1.append([])


    def on_epoch_end(self, epoch, logs = None):
        y_pred = self.model.predict(self.X_Test)
        y_pred = get_labels(y_pred)

        total = len(self.Y_Test)

        correct = 0
        for i in range(len(y_pred)):
            if y_pred[i] == self.Y_Test[i]:
                correct += 1

        accuracy = correct/total
        self.accuracies.append(correct / total)

        best = False

        if accuracy >= self.max_accuracy:
            self.max_accuracy = accuracy
            best = True


        if accuracy > 0.95:
          best = True

        for i in range(self.n_classes):
            TP = 0
            FP = 0
            TN = 0
            FN = 0

            for j in range(len(y_pred)):
                if self.Y_Test[j] == i and y_pred[j] == i:
                    TP += 1
                elif self.Y_Test[j] != i and y_pred[j] == i:
                    FP += 1
                elif self.Y_Test[j] == i and y_pred[j] != i:
                    FN += 1
                elif self.Y_Test[j] != i and y_pred[j] != i:
                    TN += 1

            sensitivity = TP / (TP + FN) if (TP + FN) > 0 else -1
            specificity = TN / (TN + FP) if (TN + FP) > 0 else -1
            precision = TP / (TP + FP) if (TP + FP) > 0 else -1
            recall = TP / (TP + FN) if (TP + FN) > 0 else -1
            accuracy = (TP + TN) / (TP + FN + TN + FP)
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else -1

            self.mat_sensitivity[i].append(sensitivity)
            self.mat_specificity[i].append(specificity)
            self.mat_precision[i].append(precision)
            self.mat_recall[i].append(recall)
            self.mat_accuracy[i].append(accuracy)
            self.mat_f1[i].append(f1)

        self.epochs.append(int(epoch+1))

        if (epoch + 1) % 10 == 0:
           from keras import backend as K
           print('learning rate changed from ', end = '')
           print(self.model.optimizer.learning_rate, end = '')
           K.set_value(self.model.optimizer.learning_rate, self.model.optimizer.learning_rate * 0.9)
           print(' to ', end = '')
           print(self.model.optimizer.learning_rate)

        if (epoch + 1) % self.save_after == 0:
            save_to_csv_file(self.save_csv_path, self.mat_sensitivity, self.mat_specificity, self.mat_precision, self.mat_recall, self.mat_accuracy, self.mat_f1, self.accuracies, self.epochs)
            self.model.save(self.model_name)
            print("max Accuracy: ", self.max_accuracy)
            if best:
                self.model.save("best_epoch" + str((epoch+1)) + "_" + self.model_name)

            pass

def save_to_csv_file(path, mat_sensitivity, mat_specificity, mat_precision, mat_recall, mat_accuracy, mat_f1, accuracies, epochs):
    mat_sensitivity = np.transpose(mat_sensitivity)
    mat_specificity = np.transpose(mat_specificity)
    mat_precision = np.transpose(mat_precision)
    mat_recall = np.transpose(mat_recall)
    mat_accuracy = np.transpose(mat_accuracy)
    mat_f1 = np.transpose(mat_f1)
    accuracies = np.reshape(accuracies, (-1,1))
    epochs = np.reshape(epochs, (-1,1))

    mat_join = np.concatenate((epochs,accuracies,mat_sensitivity, mat_specificity, mat_precision, mat_recall, mat_accuracy, mat_f1), axis = 1)

    n = mat_sensitivity.shape[1]


    col_array_sensitivity = list(range(n))
    col_array_specificity = list(range(n))
    col_array_precision = list(range(n))
    col_array_recall = list(range(n))
    col_array_accuracy = list(range(n))
    col_array_f1 = list(range(n))


    for i in range(n):
        col_array_sensitivity[i] = 'sensitivity Class ' + str(col_array_sensitivity[i])
        col_array_specificity[i] = 'specificity Class ' + str(col_array_specificity[i])
        col_array_precision[i] = 'precision Class ' + str(col_array_precision[i])
        col_array_recall[i] = 'recall Class ' + str(col_array_recall[i])
        col_array_accuracy[i] = 'accuracy Class ' + str(col_array_accuracy[i])
        col_array_f1[i] = 'f1 Class ' + str(col_array_f1[i])

    cols = ['Epoch']+['overall_accuracy']+ col_array_sensitivity + col_array_specificity + col_array_precision + col_array_recall + col_array_accuracy + col_array_f1

    mat_join = np.flip(mat_join, axis = 0)



    df = pd.DataFrame(
        columns = cols,
        data  = mat_join
    )

    df.to_csv(path, index= False)


In [ ]:
# One hot encoding
labels_train_encoded = []
for label in labels_train:
    encoding = [0,0,0,0]
    encoding[int(label)] = 1
    labels_train_encoded.append(encoding)

labels_train_encoded = np.array(labels_train_encoded)

labels_test_encoded = []
for label in labels_test:
    encoding = [0,0,0,0]
    encoding[int(label)] = 1
    labels_test_encoded.append(encoding)

labels_test_encoded = np.array(labels_test_encoded)


In [ ]:
model = tf.keras.models.Sequential()

raw_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(128,128,3),
    pooling=None
)

model.add(raw_model)

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(4, activation='softmax'))



In [ ]:

# class_weights = [2.18, 1.459, 0.719, 0.259, 7.64, 2.0459]
# [cedar rust, general scab, grey spot, healthy, serious cedar rust, serious scab]

#[healthy, general scab, serious scab, grey spot, genral cedar rust, serious cedar rust]
# class_weights = [0.259, 1.459, 2.0459, 0.719, 2.18, 7.64]



loss = tf.keras.losses.CategoricalFocalCrossentropy(
    alpha=0.25,
    gamma=0.1,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    name='categorical_focal_crossentropy'
)

loss = tf.keras.losses.CategoricalCrossentropy()

tf.keras.utils.set_random_seed(2)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
tf.keras.utils.set_random_seed(21)

model.summary()
model.compile(optimizer=opt, loss=loss, metrics = ['acc'])
# model.compile(optimizer='adam', loss=loss, metrics = ['acc'], loss_weights=class_weights)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 64)                167520    
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 batch_normalization_15 (Ba  (None, 64)                256       
 tchNormalization)                                               
                                                                 
 dense_10 (Dense)            (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization_16 (Ba  (None, 256)               1024      
 tchNormalization)                                      

In [ ]:
history = model.fit(images_train, labels_train_encoded, batch_size=128, epochs=500, validation_data=(images_test, labels_test_encoded), callbacks=[Save_Multiclass_Metrics_By_Epoch((images_test,labels_test), n_classes=4, save_after=1, save_csv_path="results_xception_part1.csv", model_name="xception_part1.keras")])


Epoch 1/500
16/16 [==============================] - 1s 20ms/step
max Accuracy:  0.590818363273453
245/245 [==============================] - 43s 95ms/step - loss: 0.8755 - acc: 0.6670 - val_loss: 0.9724 - val_acc: 0.5908
Epoch 2/500
16/16 [==============================] - 0s 11ms/step
max Accuracy:  0.8642714570858283
245/245 [==============================] - 20s 81ms/step - loss: 0.5245 - acc: 0.8058 - val_loss: 0.3642 - val_acc: 0.8643
Epoch 3/500
16/16 [==============================] - 0s 10ms/step
max Accuracy:  0.874251497005988
245/245 [==============================] - 20s 81ms/step - loss: 0.4040 - acc: 0.8545 - val_loss: 0.3070 - val_acc: 0.8743
Epoch 4/500
16/16 [==============================] - 0s 10ms/step
max Accuracy:  0.9141716566866267
245/245 [==============================] - 20s 81ms/step - loss: 0.3476 - acc: 0.8763 - val_loss: 0.2666 - val_acc: 0.9142
Epoch 5/500
16/16 [==============================] - 0s 10ms/step
max Accuracy:  0.9141716566866267
245/245 [=

In [ ]:
# from keras import backend as K
# K.set_value(model.optimizer.learning_rate, 0.00005)

In [ ]:
# model.save('drive/MyDrive/Saved Models Corn/best_epoch175_customModel_part1.keras')